# Not so random...

The success of hedge funds fundamentally boils down to the question of market efficiency. If markets are perfectly efficient, the ability for hedge funds to return above-market risk-adjusted returns should be at best a matter of luck. While the religious believers of Efficient Market Hypothesis (EMH) might justify the notable successes of the Buffets, Simons and the Griffins of this world through the law of large numbers, the majority of studies indicate prices in most markets to be, at most, weak-form efficient. While this may seem obvious, we can test this for ourselves, using a range of different methods for testing serial correlation. Using the statsmodels package, we can make use of the Durbin-Watson and Ljung Box-test to test the presence of serial correlation in a stock. In the plot below, we can perform a Ljung box-test on a year's worth of Apple stock market returns. Using this data, we see a strong argument in against market efficiency, given the Durbin-Watson Statistic shown in our test.

In [ ]:
import os
import pickle
from functools import reduce
from operator import mul

import pandas as pd
import numpy as np

from statsmodels.regression.linear_model import OLS
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn import linear_model
from sklearn.decomposition import PCA

import holoviews as hv
import hvplot
import hvplot.pandas

In [ ]:
np.random.seed(42)
hv.extension('bokeh')

In [ ]:
# There is a compatilibility issue with this library \
#and newer versions of Pandas, this is short fix to the problem, \
#if you have issues at this chunk comment it out and you should be fine.  
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as pdr

The success of hedge funds fundamentally boils down to the question of market efficiency. If markets are perfectly efficient, the ability for Hedge Funds to return above-market risk-adjusted returns should be at best a matter of luck.  While the religious believers of Efficient Market Hypothesis (EMH) might justify the notable successes of the Buffets, Simons and the Griffin's of this world through the law of large numbers, the majority of studies indicate prices in most markets to be, at most, weak-form efficient.  While this may seem obvious, we can test this for ourselves, using a range of different methods for testing serial correlation.  Using the statsmodels package, we can make use of the Durbin-Watson and Ljung Box-test to test the presence of serial correlation in a stock.  In the plot below, we can perform a Ljung box-test on a year's worth of Apple stock market returns.  Using this data, we see a strong argument in against market efficiency, given the high p-values shown in our test.  

In [ ]:
apple = pdr.robinhood.RobinhoodHistoricalReader(['AAPL'], 
                                                retry_count=3, 
                                                pause=0.1, 
                                                timeout=30, 
                                                session=None, 
                                                freq=None, 
                                                interval='day', 
                                                span='year').read().reset_index()

dw = durbin_watson(pd.to_numeric(apple.close_price).pct_change().dropna().values)
print(f'DW-statistic of {dw}')

In [ ]:
pd.Series(acorr_ljungbox(pd.to_numeric(apple.close_price).pct_change().dropna().values)[1]).hvplot.line(label="p-values at lags")

This strongly exceeds the upper-bound of the DW-statistic at the 5% level, indicating the presence of first order correlation.

The question then remains: if markets are inefficient, where is this inefficiency? This question has remained at the forefront of research for decades. Fundamentally, investors not only want to be able to quantify sources of return but also want to identify sources of potential portfolio risk. If market returns can be considered white noise, is there some trend or underlying factor which will allow us to identify and understand these risks?

The simplest of these models, Capital Asset Pricing Model (CAPM) developed by Treynor (1961), Treynor (1962), Sharpe (1964), Lintner (1965), Mossin (1966) and Black, Jensen & Scholes (1972), remains at the core of modern financial theory by providing investors with a framework in determining how the expected return of an investment is affected 
by its exposure to the systematic risk.

$$ \text{Expected Return} = r_f+β(r_m-r_f)$$

Where Expected Return is the expected returns of a share in the market, $r_{f}$, is the risk-free rate, $r_{m}$ are the returns of the market, and, $\beta$ is a coefficient computed using Ordinary Least Squares Regression, under the assumption of normally distributed errors.  
  
Under the CAPM, an asset may only earn a higher average return given an increase in exposure to a comprehensive market portfolio, as denoted by $\beta$, which should capture all systematic risk in the market. However, given that the market portfolio, which should exist as the universe of all investable assets, is not identifiable in reality, a market index is used as a proxy. While the application of CAPM is ubiquitous both in practice and in research, there exists numerous papers investigating markets around the work which critique its application over concerns over the emergence of stylized facts, the existence of cohesive market portfolios and many practical concerns over market concentration and liquidity.
  

While this set of notes will not aim to investigate the validity of the CAPM model, we will investigate the Arbitrage Pricing Theory (APT) as a segue into its implications on hedge fund construction, analysis and risk (Ross, 1976). Sadly, as discussed in the lecture recordings, the availability of public, open hedge fund data is limited, and so this module will be relying primarily on market-data, data on ETF's and famous academic datasets.

APT is a generalized framework for asset pricing that sets the expected return of an asset as a linear function of various factors, denoted below:  
  
$$ \text{Expected Value} = \beta_{0} + \beta_{1} F_{1} + ... + \beta_{n} F_{n}$$
  
While this may appear simple, given your exposure to advanced methods in Statistical Learning, the use of linear models in this application allows for computational stability and inference- crucial to many of its extensions.  

While a number of behavioral studies have been investigated in understanding non-randomness in markets, one on-going area of research has been in the use of factor models. Most factor models explore some combination of portfolio fundamentals in trying to analyze sources of non-systematic return. In seminal papers by Banz (1981) and Basu (1983), researchers explore the presence of a size- and value-effect in predicting expected returns. These factors analyze the Market Cap and PE-ratios of companies, under the APT framework, including these variables alongside the traditional market returns and risk-free rate.  
  
While research into these anomalies has varied in its findings, suggesting them a possible function of market dynamics at a point in time, studies by Lizenverg & Ramasamy (1979), Stattman (1980) and Rosenberg (1885) suggest Dividend Yield and Book-to-Market as other significant stylized facts. This research is not limited to American and European markets. In studies around the world, researchers have identified factors like momentum, cashflows, NAV and sector index as factors relevant to particular markets. Some of the most famous studies in the area of factor models has been in the Fama-French 3- and 5-Factor models. These models include market returns, size, book-to-market, operating profitability and investment.  

While the presence of these factors, many argue, provides a strong argument for the use of an exploration of statistical modelling in finance, there exist a number of counter-arguments which aim to break down the idea of just trying everything. The first argument raised by most efficiency market believers is about liquidity risk. While the size effect does indicate a negative correlation between size and expected returns, many smaller stocks are far less liquid on an exchange and, as such, present a risk to investors during times of extreme market failure. Secondly, opponents argue that many of these anomalies are temporal. In the book, The Quants, author Scott Patterson details the increasingly large leverage required by many funds towards the end of a particular trading strategies life-time as many new copy-cats enter a particular strategy. Lastly, often simple cost can limit the ability to act on a particular trade. Fundamentally, if one cannot realistically profit from a market anomaly or market inefficiency, then its ability to be realistically considered an argument in favour of market inefficiency is void.

Additionally, in the case of hedge funds, not only do these strategies need to exceed transaction costs and overcome liquidity risk in the market, but for the investor, trades must justify the cost structure of a hedge fund and the common lockup clause – which many argue presents an implied cost to the investor. While some may argue that active management ensures the pricing efficiency necessary in order to ensure passive funds can profit the reality is, from an investor point of view, Passive Funds have on average outperformed active management over a long time horizon.

For students unfamiliar with the research discussed in these notes, I would recommend reading further in your own time.  The [Podcast Freakonomics Radio](http://freakonomics.com/podcast/stupidest-thing-can-money/), has an interesting show on passive vs active investments. The show interviews Vanguage founder John C. Bogle who shares a lifetime of knowledge into running a passive fund and its growing acceptance among consumers. I would also recommend a blogpost in [Turing Finance](http://www.turingfinance.com/testing-the-efficient-market-hypothesis-with-r/) on testing market efficiency.  

# References
Fama, E. F. (1965a). The behaviour of stock market prices, Journal of Business 38, 34–105.

Fama, E. F. (1965b). Random walks in stock market prices, Financial Analysts Journal, 21, 55–9.

Fama, E. F. (1970). Efficient capital markets, a review of theory and empirical work, Journal of Finance, 25,383–417.

Fama, E. F. (1965a). The behaviour of stock market prices, Journal of Business 38, 34–105.

Fama, E. F. and French, K. R. (1988). Dividend yields and expected stock returns, Journal of Financial Economics, 22(1), 3-25.

Jegadeesh, N. and Titman, S. (1993). Returns to buying winners and selling losers: Implications for stock market efficiency, Journal of Finance, 48, 65-91.

Jensen, M. (1978). Some anomalous evidence regarding Market Efficiency, Journal of Financial Economics, 6, 95 –102.

Lo, A. W. and MacKinlay, C. A. (1988). Stock market prices do not follow random walks, evidence from a simple specification test, Review of Financial Studies, Oxford University Press for Society for Financial Studies, 1(1), 41-66.

Markowitz, H. M. (1952), Portfolio selection, The Journal of Finance, 7 (1), 77-91.

Ross, S. (1976). The arbitrage theory of capital asset pricing, Journal of Economic Theory, 13 (2), 341 – 360.

Sharpe, W. (1964). Capital Asset Prices: A theory of market equilibrium under conditions of risk, The Journal of Finance, 19 (3s), 425 – 442.
